In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

import numpy
import pandas as pd

## 预备知识：关于矩阵的奇异值和SVD实战

In [ ]:

## 在这个示例中，矩阵 A 是一个3x3的矩阵，但是由于它的行和列线性相关，因此它的秩为2，小于行数和列数中的较小者（3）。因此，这个矩阵会有多个非零奇异值。

import numpy as np

# 创建一个3x3的非满秩矩阵
A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# 计算奇异值分解
U, S, V = np.linalg.svd(A)

print("左奇异向量：\n", U)
print("奇异值：", S)
print("右奇异向量：\n", V)

In [ ]:
# config data
N = 3  # num of SVD feature. either 2 or 3
def generate_component_list(num):
    component_list = ["component_" + str(i) for i in range(1, num + 1)]
    return component_list
    
components_label = generate_component_list(N)
print(components_label)

## SVD做语义分析分类

In [ ]:
corpus = ["Python is popular in machine learning",
         "Distributed system is important in big data analysis",
        "Machine learning is theoretical foundation of data mining",
        "Learning Python is fun",
        "Playing soccer is fun",
        "Many data scientists like playing soccer",
        "Chinese men's soccer team failed again",
        "Thirty two soccer teams enter World Cup finals"]

vectorizer = CountVectorizer(min_df=1, stop_words="english")
data = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

pd.DataFrame(data.toarray(), index=corpus, columns=vectorizer.get_feature_names_out()).head(10)

In [ ]:
"""Singular value decomposition and LSA"""

model = TruncatedSVD(N) # 使用奇异值分解的方法，建立一个降维模型，参数2：指定要保留的主成分数量，即降维后的特征数量。
data_n = model.fit_transform(data)  #将原始数据输入到 TruncatedSVD 模型中进行拟合和变换操作。这将返回降维后的数据 data_n，其中特征数量已经被减少到 2 维。

# 对降维后的数据 data_n 进行归一化处理。Normalizer 是用来对数据进行归一化的类，参数 copy=False 表示对数据进行归一化时不会创建副本，而是直接在原始数据上进行操作。
#.fit_transform() 方法将归一化操作应用于数据，并返回归一化后的数据 data_n。
data_n = Normalizer(copy=False).fit_transform(data_n)

#data_n 是经过降维和归一化处理后的数据，它是一个二维数组，
#每行代表一个样本，每列代表一个特征。
#本例降维操作将特征数量减少到了 2，所以 data_n 中每个样本都表示为一个包含两个特征值的向量。
#在这种情况下，每个样本的2个特征可能代表数据中的某种抽象性质或潜在结构，这些特征是根据原始数据中的各种特征通过降维技术提取出来的。因此，data_n 中的每个样本可以看作是一个经过降维和归一化处理后的数据点，其具体意义可能与原始数据的特征不同，但仍然保留了数据的主要结构或关系。
data_n

In [ ]:
pd.DataFrame(data_n, index = corpus, columns = components_label)

In [ ]:
def plot_2dim(data_n):
    xs = data_n[:,0]
    ys = data_n[:,1]
    import matplotlib.pyplot as plt
    import seaborn as sns
    %matplotlib inline
    
    plt.figure(figsize=(4,4))
    ax = plt.gca()
    ax.set_xlim([-1, 2])
    ax.set_ylim([-1, 2])
    plt.scatter(xs, ys)
    plt.xlabel('First principal component')
    plt.ylabel('Second principal component')
    plt.title('Plot of points agains LSA principal components')
    plt.show()

def plot_3dim(data_n):
    from mpl_toolkits import mplot3d
    # 假设 data_n 有三列数据
    # 提取数据的三个维度
    x = data_n[:, 0]
    y = data_n[:, 1]
    z = data_n[:, 2]
    
    # 创建一个三维图形对象
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # 绘制三维散点图
    ax.scatter(x, y, z)
    
    # 设置坐标轴标签和标题
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    ax.set_title('3D Scatter Plot')
    
    # 显示图形
    plt.show()

In [ ]:
if N == 2:
    plot_2dim(data_n)
elif N == 3:
    plot_3dim(data_n)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 假设 data_n 有多于两列的数据
# 进行 PCA 降维，保留前两个主成分
pca = PCA(n_components=2)
data_2d = pca.fit_transform(data_n)

# 绘制散点图
plt.figure(figsize=(6, 6))
plt.scatter(data_2d[:, 0], data_2d[:, 1])
plt.xlabel('First principal component')
plt.ylabel('Second principal component')
plt.title('Plot of points against first two PCA components')
plt.show()

In [ ]:
similarity = numpy.asarray(numpy.asmatrix(data_n) * numpy.asmatrix(data_n).T)
pd.DataFrame(similarity, index = corpus, columns = corpus).head(10)

In [ ]:
sns.heatmap(similarity,cmap='Reds')

In [ ]:
pd.DataFrame(model.components_,index=['component_1','component_2'],columns=vectorizer.get_feature_names_out()).T